### Part 1: Scraping Neighborhood data from Wikipedia
Using BeautifulSoup package

In [20]:
import numpy as np
import pandas as pd
import urllib3
from bs4 import BeautifulSoup

import re

In [21]:
http = urllib3.PoolManager()
r = http.request('GET', 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


**Read wikipedia url content**

In [22]:
# get raw data from web
bs = BeautifulSoup(r.data, 'html.parser')
table_body=bs.find('tbody')
#print (table_body)

#declare some list as global
neighbor_list = []
columns = []
nb_data = []

tr_rows = bs.find_all('tr')
print (tr_rows)

[<tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>, <tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>, <tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>, <tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>, <tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>, <tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>, <tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>, <tr>
<td>M6A</td>
<td><a href="/wiki/North_York" ti

**Get specific data inside the html content of url**

In [23]:
initial=True
#print (len(tr_rows))

for tr_row in tr_rows:
    #print (tr_row)
    #print ("-----------------")
    if initial==True:
        col=[]
        th=tr_row.find_all('th')
        #print (th)
        for th_row in th:
            #print (th_row.text)
            col.append(str(th_row.text).rstrip())
        
        initial=False
        
    else:
        data=[]
        td=tr_row.find_all('td')
        #print (td)
        for td_row in td:
            #print (td_row.text)
            data.append(str(td_row.text).rstrip())
            
        #append data into list
        nb_data.append(data)
    
        #print ("*************")
        #print ("col=",col)
        #print ("data=",data)
        #print ("*************")
    
columns = col.copy()

print ("###########################")
print ("columns = ",columns)
print ("nb_data = ",nb_data)
print ("###########################")
    
print ("Completed for web scraping!")

###########################
columns =  ['Postcode', 'Borough', 'Neighbourhood']
nb_data =  [['M1A', 'Not assigned', 'Not assigned'], ['M2A', 'Not assigned', 'Not assigned'], ['M3A', 'North York', 'Parkwoods'], ['M4A', 'North York', 'Victoria Village'], ['M5A', 'Downtown Toronto', 'Harbourfront'], ['M5A', 'Downtown Toronto', 'Regent Park'], ['M6A', 'North York', 'Lawrence Heights'], ['M6A', 'North York', 'Lawrence Manor'], ['M7A', "Queen's Park", 'Not assigned'], ['M8A', 'Not assigned', 'Not assigned'], ['M9A', 'Etobicoke', 'Islington Avenue'], ['M1B', 'Scarborough', 'Rouge'], ['M1B', 'Scarborough', 'Malvern'], ['M2B', 'Not assigned', 'Not assigned'], ['M3B', 'North York', 'Don Mills North'], ['M4B', 'East York', 'Woodbine Gardens'], ['M4B', 'East York', 'Parkview Hill'], ['M5B', 'Downtown Toronto', 'Ryerson'], ['M5B', 'Downtown Toronto', 'Garden District'], ['M6B', 'North York', 'Glencairn'], ['M7B', 'Not assigned', 'Not assigned'], ['M8B', 'Not assigned', 'Not assigned'], ['M9B', 'Eto

**Get into dataframe the Neighborhood data table in wiki**

In [24]:
#columns = col.copy()
#print (columns)
df_toronto = pd.DataFrame(columns=columns)
df_toronto.head()

len_eq_3 = [(len(item) == 3) for item in nb_data]
#len_eq_3

toronto_neiborhoods_list=[]
for i, cond in zip(range(0,len(len_eq_3)),len_eq_3):
    #print (i,cond)
    if cond==True:
        toronto_neiborhoods_list.append(nb_data[i])

#print ("Processed data: toronto_neiborhoods_list = ",toronto_neiborhoods_list)

#toronto_neighborhoods = nb_data[1:-5]
#df for 
df_toronto = pd.DataFrame(data=toronto_neiborhoods_list, columns=columns)

df_toronto.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [25]:
# only get the Borough having value, ignore 'Not assigned'
df_toronto_1 = df_toronto[(df_toronto['Borough']!='Not assigned')].reset_index(drop=True)
df_toronto_1.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [26]:
#Neighbourhood have 'Not assigned' value replace the same name of Borough
for i,item in zip(range(0,len(df_toronto_1['Neighbourhood'])),df_toronto_1['Neighbourhood']):
    #print (i,item)
    if item=='Not assigned':
        df_toronto_1.loc[i,'Neighbourhood'] = df_toronto_1.loc[i,'Borough']
        
df_toronto_1.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [27]:
#reset index
df_toronto_1.reset_index(drop=True, inplace=True)
df_toronto_1.head()
print (df_toronto_1.columns)

Index(['Postcode', 'Borough', 'Neighbourhood'], dtype='object')


- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods **separated with a comma as shown in row 11** in the above table.

In [28]:

df_toronto_2 = df_toronto_1.groupby(by=['Postcode','Borough']).agg(lambda x: ','.join(x))
df_toronto_2.reset_index(level=['Postcode','Borough'], inplace=True)

print (df_toronto_2.columns)
df_toronto_2.head(10)

Index(['Postcode', 'Borough', 'Neighbourhood'], dtype='object')


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [29]:
df_toronto_2.shape

(103, 3)

**COMPLETE DATA SCRAPING AND PROCESSING!!!**